In [ ]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import model_selection


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn import datasets, neighbors, metrics, tree, svm, preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import ensemble

# Calculates the per-class accuracy given predicted and true output labels.
def class_accs(y_pred, y_true):
    acc0 = ((y_pred == y_true) & (y_true == 0)).sum() / (y_true == 0).sum()
    acc1 = ((y_pred == y_true) & (y_true == 1)).sum() / (y_true == 1).sum()
    return acc0, acc1

# Prints a summary of performance metrics given predicted and true output labels. (source: pract3)
def print_metrics(y_pred, y_true):
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
    f1 = metrics.f1_score(y_true, y_pred, average=None)
    acc = metrics.accuracy_score(y_true, y_pred)
    #acc0, acc1 = class_accs(y_pred, y_true)
    print(f'\tF1 = {f1}')
    print(f'\tAccuracy = {acc}')
    #print(f'\t\tclass 0: {acc0}')
    #print(f'\t\tclass 1: {acc1}')
    



In [ ]:
%%time

# load labels
labels = pd.read_csv('../data/Lauren/Labels.csv')
#labels.head() # to display the first 5 lines of loaded data

# load data
df = pd.read_csv('../data/Lauren/500_PBMC_3p_LT_Chromium_X_metrics_summary.csv') # takes about 5min



CPU times: user 5min 11s, sys: 6.49 s, total: 5min 18s
Wall time: 5min 21s


In [ ]:
def filter_info(df, labels, filter_on, amount_higher_than):
    ## Filter info
    classes_count = labels.groupby(filter_on).count()
    #print(classes_count)
    # the count is in last column: classes_count[classes_count.columns[-1]
    classes_to_keep = list(classes_count[classes_count[classes_count.columns[-1]] >= amount_higher_than].index)
    #print(f"{len(classes_to_keep)} classes to keep: {classes_to_keep}")
    
    return classes_to_keep

classes_to_keep = filter_info(df, labels, "cluster", 10)


In [ ]:
keep_indices = labels["cluster"].isin(classes_to_keep)
labels = labels[keep_indices]
df = df[keep_indices]

In [ ]:
print(f"Classes left: {filter_info(df, labels, 'Class', 10)}")

Classes left: ['GABAergic', 'Glutamatergic', 'Non-Neuronal']


In [ ]:
# filter out Endothelial because barely any entries
#keep_indices = labels["Class"].isin(['GABAergic', 'Glutamatergic', 'Non-Neuronal'])
#labels = labels[keep_indices]
#df = df[keep_indices]

# remove names of cells
df = df.drop(columns=["Unnamed: 0"])
# only keep most specific classification (for now)
labels = labels.drop(columns = ["Class", "Subclass"]).values.ravel()

In [ ]:
labels

array(['Vip Arhgap36 Hmcn1', 'Lamp5 Lsp1', 'Lamp5 Lsp1', ...,
       'Pvalb Gpr149 Islr', 'L5 PT VISp C1ql2 Cdh13', 'Sst Calb2 Pdlim5'],
      dtype=object)

In [ ]:
#%%time GOOD RESULT DONT DELETE
#size = df.shape[0]
#train_size = int(size/10)
#X_train = df[:train_size]
#y_train = labels[:train_size]
#X_test = df[train_size:2*train_size]
#y_test = labels[train_size:2*train_size]

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
# kies meer random: 
skf = StratifiedKFold(n_splits=2, random_state=1337, shuffle=True)
print(f"Number of splitting iterations in the cross-validator skf: {skf.get_n_splits(df)}")
print(skf)
print()

for train_index, test_index in skf.split(df, labels):
    # get train and test set
    print(train_index)
    X_train, X_test = df.take(train_index), df.take(test_index)
    y_train, y_test = labels[train_index], labels[test_index]
    
    # 1vRest training
    print("TRAIN:", train_index, "TEST:", test_index)
    print("Start fitting")
    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train, y_train)
    
    # predicting
    print("Start predicting")
    y_pred_lin_clf = lin_clf.predict(X_test)
    
    # metrics
    print("Calculating metrics")
    f1 = metrics.f1_score(y_test, y_pred_lin_clf, average=None)
    acc = metrics.accuracy_score(y_test, y_pred_lin_clf)
    
    unique_labels_df = pd.DataFrame(pd.Series(y_test).unique())
    f1_df = pd.DataFrame(f1)
    print(pd.concat([unique_labels_df, f1_df], axis=1, keys=['cluster', 'f1_per_cluster']))
    print(acc)
    
    # just do 1 iteration
    break
    
# duurt super lang
#from sklearn import model_selection
#print(model_selection.cross_validate(svm.SVC(kernel = 'linear'), X_train, y_train))

Number of splitting iterations in the cross-validator skf: 2
StratifiedKFold(n_splits=2, random_state=1337, shuffle=True)

[    0     3     4 ... 12777 12778 12780]
TRAIN: [    0     3     4 ... 12777 12778 12780] TEST: [    1     2     5 ... 12773 12774 12779]
Start fitting


/home/iwijn/ml/machine-learning-project/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Start predicting
Calculating metrics
                   cluser f1_per_cluster
                        0              0
0              Lamp5 Lsp1       0.909091
1          Sst Tac1 Tacr3       0.858586
2             Lamp5 Krt73       0.951952
3        Vip Igfbp6 Car10       0.833333
4       Lamp5 Plch2 Dock5       0.992571
..                    ...            ...
88    L6 CT ALM Nxph2 Sla       0.805195
89     Pvalb Sema3e Kank4       0.947368
90   L5 IT VISp Whrn Tox2       0.773723
91  L6b VISp Col8a1 Rxfp1       0.611111
92           L6b VISp Crh       0.793103

[93 rows x 2 columns]
0.9023626975434205
Start predicting


In [ ]:
# https://medium.com/synthesio-engineering/precision-accuracy-and-f1-score-for-multi-label-classification-34ac6bdfb404
print(f"F1 micro-average: {metrics.f1_score(y_test, y_pred_lin_clf, average='micro')}")
print(f"F1 macro-average: {metrics.f1_score(y_test, y_pred_lin_clf, average='macro')}")
print(f"F1 weighted-average: {metrics.f1_score(y_test, y_pred_lin_clf, average='weighted')}")
#acc = metrics.accuracy_score(y_test, y_pred_lin_clf)

F1 micro-average: 0.9023626975434204
F1 macro-average: 0.8430137881777549
F1 weighted-average: 0.9004396258585396


In [ ]:
# export f1 scores (file in results/ folder)
full_f1 = pd.concat([unique_labels_df, f1_df], axis=1, keys=['cluster', 'f1_per_cluster'])
full_f1.to_csv("all_f1_scores_flat.csv", index=False)

In [ ]:
# checks
for train_index, test_index in skf.split(df, labels):
    # get train and test set
    X_train, X_test = df.take(train_index), df.take(test_index)
    y_train, y_test = labels[train_index], labels[test_index]
    
    print(X_train)
    print(y_train)
    
    break

       0610005C13Rik  0610006L08Rik  0610007P14Rik  0610009B22Rik  \
0                  0              0             79            145   
3                  0              0            115              0   
4                  0              0            402            208   
6                  0              0             82            154   
11                 0              0             97              0   
...              ...            ...            ...            ...   
12826              0              0            264            143   
12827              0              0             45             82   
12828              0              0             88             79   
12829              0              0             81            105   
12831              0              0            112            184   

       0610009E02Rik  0610009L18Rik  0610009O20Rik  0610010B08Rik  \
0                  1             46            123              0   
3                  0             

In [ ]:
#1v1
# no good results
#print("Start fitting")
#clf = svm.SVC(decision_function_shape='ovo')
#clf.fit(X_train, y_train)
#
#print("Start predicting")
#y_pred_clf = clf.predict(X_test)

In [ ]:
# 1vRest
#print("Start fitting")
#lin_clf = svm.LinearSVC()
#lin_clf.fit(X_train, y_train)
#
#print("Start predicting")
#y_pred_lin_clf = lin_clf.predict(X_test)
#print("Done")

Start fitting


/home/iwijn/ml/machine-learning-project/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Start predicting


In [ ]:
#print("Calculating metrics")
#f1 = metrics.f1_score(y_test, y_pred, average=None)
#acc = metrics.accuracy_score(y_test, y_pred)
#
#unique_labels_df = pd.DataFrame(pd.Series(y_test).unique())
#f1_df = pd.DataFrame(f1)
#print(pd.concat([unique_labels_df, f1_df], axis=1, keys=['cluser', 'f1_per_cluster']))
#print(acc)

#  Problem: not all lables were included in training part of dataset => solution: larger and random

Calculating metrics
                      cluser f1_per_cluster
                           0              0
0       L2/3 IT VISp Adamts2       0.000000
1         L2/3 IT VISp Agmat       0.829268
2           L4 IT VISp Rspo1       0.934673
3   L5 IT VISp Hsd11b1 Endou       0.941176
4    L6 IT VISp Penk Col27a1       0.978102
..                       ...            ...
83               Lamp5 Krt73       0.888889
84                       NaN       0.866667
85                       NaN       0.800000
86                       NaN       0.000000
87                       NaN       0.333333

[88 rows x 2 columns]
0.8143525741029641
